In [ ]:
!pip uninstall -y tensorflow

In [3]:
!pip install tensorflow

  Using cached https://files.pythonhosted.org/packages/04/7e/a484776c73b1431f2b077e13801531e966113492552194fe721e6ef88d5d/tensorflow-1.10.1-cp36-cp36m-manylinux1_x86_64.whl
You are using pip version 9.0.3, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [20]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import csv
import IPython
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import tensorflow as tf
import tensorflow_probability as tfp
import warnings

from tensorflow_probability import edward2 as ed

plt.style.use('ggplot')

In [5]:
def build_toy_dataset(N, w, noise_std=0.1):
    D = len(w)
    x = np.random.randn(N, D)
    y = np.dot(x, w) + np.random.normal(0, noise_std, size=N)
    return x, y

N = 40  # number of data points
D = 10  # number of features

w_true = np.random.randn(D)
X_train, y_train = build_toy_dataset(N, w_true)
X_test, y_test = build_toy_dataset(N, w_true)

In [10]:
X_train.shape, y_train.shape

((40, 10), (40,))

\begin{align*}
\mathbf{w} &\sim \text{Normal}(\mathbf{0}, \sigma^2 \mathbf{I}) \\
b &\sim \text{Normal}(0, \sigma_{b}^2) \\
\end{align*}

In [24]:
from tensorflow_probability import edward2 as ed
def model(X):
    w = ed.Normal(loc=0., scale=1., sample_shape=X.shape[1], name="w")
    y = ed.Normal(loc=tf.tensordot(X, w, [[1], [0]]), scale=0.1, name="y")
    return y

log_joint = ed.make_log_joint_fn(model)

X = tf.random_normal([3, 2])
w_value = tf.random_normal([2])
y_value = tf.random_normal([3])
output = log_joint(X, w=w_value, y=y_value)

In [26]:
qw = ed.Normal(loc=tf.get_variable("qw/loc", [D]),
            scale=tf.nn.softplus(tf.get_variable("qw/scale", [D])))
qb = ed.Normal(loc=tf.get_variable("qb/loc", [1]),
            scale=tf.nn.softplus(tf.get_variable("qb/scale", [1])))

In [27]:
inference = ed.KLqp({w: qw, b: qb}, data={X: X_train, y: y_train})

AttributeError: module 'tensorflow_probability.python.edward2' has no attribute 'KLqp'

In [25]:
log_joint

<function tensorflow_probability.python.edward2.program_transformations.make_log_joint_fn.<locals>.log_joint_fn(*args, **kwargs)>

In [23]:
output

<tf.Tensor 'add_1:0' shape=() dtype=float32>

In [18]:
sigma_w = tf.get_variable("sigma_w", [1.0])

In [19]:
sigma_w

<tf.Variable 'sigma_w:0' shape=(1,) dtype=float32_ref>

In [ ]:
from edward.models import Normal

X = tf.placeholder(tf.float32, [N, D])
w = Normal(loc=tf.zeros(D), scale=tf.ones(D))
b = Normal(loc=tf.zeros(1), scale=tf.ones(1))
y = Normal(loc=ed.dot(X, w) + b, scale=tf.ones(N))

In [ ]:
ed.mo

In [13]:
def linear_model(exog, sig_w = 1.0, sig_b = 1.0):
    X = tf.placeholder

In [ ]:
def linear_mixed_effects_model(features):
  # Set up fixed effects and other parameters.
    intercept = tf.get_variable("intercept", [])  # alpha in eq
    
    effect_service = tf.get_variable("effect_service", [])  # beta in eq
    
    stddev_students = tf.exp(
      tf.get_variable("stddev_unconstrained_students", []))  # sigma in eq
    stddev_instructors = tf.exp(
      tf.get_variable("stddev_unconstrained_instructors", [])) # sigma in eq
    stddev_departments = tf.exp(
      tf.get_variable("stddev_unconstrained_departments", [])) # sigma in eq

  # Set up random effects.
    effect_students = ed.MultivariateNormalDiag(
      loc=tf.zeros(num_students),
      scale_identity_multiplier=stddev_students,
      name="effect_students")
    
    effect_instructors = ed.MultivariateNormalDiag(
      loc=tf.zeros(num_instructors),
      scale_identity_multiplier=stddev_instructors,
      name="effect_instructors")
    
    effect_departments = ed.MultivariateNormalDiag(
      loc=tf.zeros(num_departments),
      scale_identity_multiplier=stddev_departments,
      name="effect_departments")

  # Set up likelihood given fixed and random effects.
  # Note we use `tf.gather` instead of matrix-multiplying a design matrix of
  # one-hot vectors. The latter is memory-intensive if there are many groups.
    ratings = ed.Normal(
        loc=(effect_service * features["service"] +
           tf.gather(effect_students, features["students"]) +
           tf.gather(effect_instructors, features["instructors"]) +
           tf.gather(effect_departments, features["departments"]) +
           intercept),
      scale=1.,
      name="ratings")  
    return ratings

# Wrap model in a template. All calls to the model template will use the same
# TensorFlow variables.
model_template = tf.make_template("model", linear_mixed_effects_model)